# Implement Page Rank Algorithm. (Use python or beautiful soup for implementation).

In [ ]:
import numpy as np
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch links from each URL
def fetch_links(url):
    """
    Takes a URL as input, fetches its HTML content, and extracts all absolute hyperlinks (starting with 'http').
    Returns:
        A set of unique links found on the given URL.
    """
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = set()  # Using a set to avoid duplicate links

        # The soup.find_all('a', href=True) command finds all <a> tags in the HTML 
        # that have an href attribute (which is where links are typically stored).

        # For each <a> tag found, the href attribute (the link) is extracted using link.get('href') 
        # and stored in the href variable.

        for link in soup.find_all('a', href=True):
            href = link.get('href')
            if href and href.startswith('http'):  # Filtering absolute links only
                links.add(href)
        
        # The code checks if the href is not empty (if href) and 
        # whether it starts with "http" (href.startswith('http')). 
        # This ensures that only absolute URLs (those beginning with "http") 
        # are added to the links set, ignoring relative links.

        # If the link meets the criteria, it is added to the links set using links.add(href).

        
        return links
    except Exception as e:
        print(f"Error fetching links from {url}: {e}")
        return set()

# Step 2: Build the web graph as a dictionary of URLs with their outgoing links
def build_web_graph(urls):
    """
    Constructs a dictionary where each URL points to a set of outgoing links from that URL.
    Args:
        urls: A list of URLs to analyze.
    Returns:
        A dictionary representing the web graph.
    """
    web_graph = {}
    for url in urls:
        links = fetch_links(url)
        web_graph[url] = links
    return web_graph

# Step 3: Create the adjacency matrix from the web graph
def create_adjacency_matrix(web_graph):
    """
    Converts the web graph dictionary into an adjacency matrix, where each element at (i, j) represents
    the link probability from URL i to URL j.
    Returns:
        adjacency_matrix: The transition matrix used for PageRank.
        nodes: A list of URLs representing each index in the adjacency matrix.
    """
    nodes = list(web_graph.keys())  # List of all nodes (URLs)
    n = len(nodes)
    node_to_index = {node: i for i, node in enumerate(nodes)}
    adjacency_matrix = np.zeros((n, n))  # Initializing a square matrix of size n x n

    for i, url in enumerate(nodes):
        links = web_graph[url]
        if links:
            for link in links:
                if link in node_to_index:
                    j = node_to_index[link]  # Mapping URL to index
                    adjacency_matrix[i, j] = 1 / len(links)
        else:
            # Handling dangling nodes (pages with no outbound links)
            adjacency_matrix[i, :] = 1 / n  # Equal probability for each page
    
    return adjacency_matrix, nodes

# Step 4: Calculate PageRank using the adjacency matrix
def calculate_pagerank(adjacency_matrix, alpha=0.85, max_iter=100, tol=1.0e-6):
    """
    Uses the power iteration method to compute the PageRank vector.
    Args:
        adjacency_matrix: The transition matrix for web pages.
        alpha: The damping factor (probability of random jump).
        max_iter: The maximum number of iterations.
        tol: Convergence tolerance.
    Returns:
        A numpy array containing the PageRank scores for each URL.
    """
    n = adjacency_matrix.shape[0]
    pagerank = np.ones(n) / n  # Start with an equal probability distribution
    teleport = np.ones(n) / n  # Teleportation vector to handle dead-ends and random jumps

    for _ in range(max_iter):
        new_pagerank = alpha * adjacency_matrix @ pagerank + (1 - alpha) * teleport
        # Check if the algorithm has converged
        if np.linalg.norm(new_pagerank - pagerank, 1) < tol:
            break
        pagerank = new_pagerank
    
    return pagerank

# List of URLs to analyze
urls = [
    "https://example.com/page1",
    "https://example.com/page2",
    "https://example.com/page3"
]

# Step 1: Build the web graph from the list of URLs
web_graph = build_web_graph(urls)

# Step 2: Create the adjacency matrix from the web graph
adjacency_matrix, nodes = create_adjacency_matrix(web_graph)

# Step 3: Calculate PageRank
pagerank_values = calculate_pagerank(adjacency_matrix)

# Display the PageRank values for each URL
print("PageRank values:")
for i, node in enumerate(nodes):
    print(f"{node}: {pagerank_values[i]:.4f}")

# Input Web Graph:

# A -> B, C
# B -> C
# C -> (no links)

# Step 1: Fetch Links (from each URL):

# A links to: B, C
# B links to: C
# C links to: (none)


# Step 2: Build Web Graph:


# web_graph = {
#     'A': {'B', 'C'},
#     'B': {'C'},
#     'C': set()
# }


# Step 3: Create Adjacency Matrix:

# The create_adjacency_matrix function:
# Converts the web graph dictionary into a list of nodes (URLs).
# Creates a square matrix (n x n).
# For each URL:
# If it links to another URL, it assigns 1 / number_of_links in the corresponding matrix position.
# If a URL has no outgoing links, it assigns equal probability for each link to all other pages.

# Nodes = ['A', 'B', 'C']
# adjacency_matrix = [
#     [0, 0, 1/3],
#     [0.5, 0, 1/3],
#     [0.5, 1, 1/3]
# ]

# Step 4: Calculate PageRank:

# Initialize:

# PR(A)=PR(B)=PR(C)= 1/3


# Apply the formula iteratively:
# After one iteration:
# PR(A)=0.4333,PR(B)=0.4333,PR(C)=0.4333

# Repeat until convergence. Final values after convergence:

# PR(A)=0.4333,PR(B)=0.4333,PR(C)=0.4333




PageRank values:
https://example.com/page1: 0.0500
https://example.com/page2: 0.0500
https://example.com/page3: 0.0500
